In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


#Importing Linear Regression and pandas, packages
import numpy as np
import pandas as pd
import matplotlib
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,165 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,264 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()



In [ ]:
#Reading in the csv file

from pyspark import SparkFiles

url = "https://raw.githubusercontent.com/Andrewli1212/project-4-Italy-Housing/main/Output/sales.csv"

spark.sparkContext.addFile(url)

file_path = "file://" + SparkFiles.get("sales.csv")

df = spark.read.csv(file_path, header=True)
#print(df.columns)
df.show()



+--------+--------------------+--------------------+---------+----------+--------------+------------------+---------+--------+---------+-----+-------------+--------+-------------------+-------+---------+-------+---------+-----------------+-----------+-------------+------+-------------+--------------+------------+-------+----+-----+---------------+---------+-----+-----+
|  region|                city|             quarter|    price|  datetime|parking_spaces|bathrooms_per_room|bathrooms|bedrooms|top_floor|state|energy_rating|sea_view|centralized_warming|surface|furnished|balcony|TV_system|external_exposure|optic_fiber|electric_gate|cellar|common_garden|private_garden|alarm_system|doorman|pool|villa|entire_property|apartment|attic|lofts|
+--------+--------------------+--------------------+---------+----------+--------------+------------------+---------+--------+---------+-----+-------------+--------+-------------------+-------+---------+-------+---------+-----------------+-----------+-----

In [ ]:
# Create a temporary view of the DataFrame.
df.createOrReplaceTempView('homesales')

In [ ]:
# Fidning the average price of a home in each city

from pyspark.sql.functions import year

query = (
    "SELECT city, ROUND(AVG(price), 0) AS avg_price "
    "FROM homesales "
    "GROUP BY city "

    )

#print(query)
spark.sql(query).show()

+--------------------+---------+
|                city|avg_price|
+--------------------+---------+
|             Palermo| 180351.0|
|             Codogno|  99500.0|
|       Villa Estense| 103000.0|
|Sant'Agata sul Sa...| 238000.0|
|    Quattro Castella| 298333.0|
|            Piombino| 160000.0|
|   Piano di Sorrento| 440000.0|
|           Spadafora|  94333.0|
|     Forte dei Marmi| 986015.0|
|           Viareggio| 479322.0|
|             Vignate| 318000.0|
|          Roccavione| 135000.0|
|              Spinea| 244815.0|
|San Giovanni in P...| 301642.0|
|         Bentivoglio| 320000.0|
|     Castell'Arquato| 390000.0|
|              Giarre| 103024.0|
|  Riva Presso Chieri| 195000.0|
|      Cassano d'Adda| 181500.0|
|              Cameri| 297500.0|
+--------------------+---------+
only showing top 20 rows



In [ ]:
# Fidning the average price of a home per bedroom

from pyspark.sql.functions import year

query = (
    "SELECT bedrooms, ROUND(AVG(price), 0) AS avg_price "
    "FROM homesales "
    "GROUP BY bedrooms "
    "ORDER BY bedrooms ASC"

    )


#print(query)
spark.sql(query).show()

+--------+---------+
|bedrooms|avg_price|
+--------+---------+
|    NULL|      1.0|
|     1.0| 245029.0|
|     2.0| 214378.0|
|     3.0| 287139.0|
|     4.0| 366531.0|
|     5.0| 450685.0|
+--------+---------+



In [ ]:
# Fidning the average price of a home in the year 2022

from pyspark.sql.functions import year

query = (
    "SELECT bedrooms, ROUND(AVG(price), 0) AS avg_price "
    "FROM homesales "
    "WHERE year(datetime) = 2022 "
    "GROUP BY bedrooms "
    "ORDER BY bedrooms ASC "
    )


#print(query)
spark.sql(query).show()

+--------+---------+
|bedrooms|avg_price|
+--------+---------+
|     1.0|1059088.0|
|     2.0| 565998.0|
|     3.0| 306555.0|
|     4.0| 470300.0|
|     5.0| 538673.0|
+--------+---------+



In [ ]:
# Fidning the average price of a home in the year 2022

from pyspark.sql.functions import year

query = (
    "SELECT bathrooms, ROUND(AVG(price), 0) AS avg_price "
    "FROM homesales "
    "WHERE year(datetime) = 2022 "
    "GROUP BY bathrooms "
    "ORDER BY bathrooms ASC "
    )


#print(query)
spark.sql(query).show()

+---------+---------+
|bathrooms|avg_price|
+---------+---------+
|      1.0| 392578.0|
|      2.0| 427902.0|
|      3.0| 566692.0|
|      5.0|8784500.0|
|      6.0|3442500.0|
|      7.0| 824000.0|
|      8.0|4161000.0|
|      9.0|3031000.0|
+---------+---------+



In [ ]:
#Testing Cache Cells
print(spark.catalog.isCached('homesales'))
start_time = time.time()

query = (
    "SELECT city, ROUND(AVG(price), 0) AS avg_price "
    "FROM homesales "
    "GROUP BY city "

    )

spark.sql(query).show()

print("--- %s seconds --- non cached" % (time.time() - start_time))

False
+--------------------+---------+
|                city|avg_price|
+--------------------+---------+
|             Palermo| 180351.0|
|             Codogno|  99500.0|
|       Villa Estense| 103000.0|
|Sant'Agata sul Sa...| 238000.0|
|    Quattro Castella| 298333.0|
|            Piombino| 160000.0|
|   Piano di Sorrento| 440000.0|
|           Spadafora|  94333.0|
|     Forte dei Marmi| 986015.0|
|           Viareggio| 479322.0|
|             Vignate| 318000.0|
|          Roccavione| 135000.0|
|              Spinea| 244815.0|
|San Giovanni in P...| 301642.0|
|         Bentivoglio| 320000.0|
|     Castell'Arquato| 390000.0|
|              Giarre| 103024.0|
|  Riva Presso Chieri| 195000.0|
|      Cassano d'Adda| 181500.0|
|              Cameri| 297500.0|
+--------------------+---------+
only showing top 20 rows

--- 1.2399306297302246 seconds --- non cached


In [ ]:
# Cache the the temporary table home_sales.

spark.catalog.cacheTable("homesales")

In [ ]:
#Check if the table is cached.
spark.catalog.isCached('homesales')

True

In [ ]:
# Using the cached data, run the query that filters out the average price by city

start_time = time.time()

query = (
    "SELECT city, ROUND(AVG(price), 0) AS avg_price "
    "FROM homesales "
    "GROUP BY city "

    )

spark.sql(query).show()


print("--- %s seconds --- cached" % (time.time() - start_time))

+--------------------+---------+
|                city|avg_price|
+--------------------+---------+
|             Palermo| 180351.0|
|             Codogno|  99500.0|
|       Villa Estense| 103000.0|
|Sant'Agata sul Sa...| 238000.0|
|    Quattro Castella| 298333.0|
|            Piombino| 160000.0|
|   Piano di Sorrento| 440000.0|
|           Spadafora|  94333.0|
|     Forte dei Marmi| 986015.0|
|           Viareggio| 479322.0|
|             Vignate| 318000.0|
|          Roccavione| 135000.0|
|              Spinea| 244815.0|
|San Giovanni in P...| 301642.0|
|         Bentivoglio| 320000.0|
|     Castell'Arquato| 390000.0|
|              Giarre| 103024.0|
|  Riva Presso Chieri| 195000.0|
|      Cassano d'Adda| 181500.0|
|              Cameri| 297500.0|
+--------------------+---------+
only showing top 20 rows

--- 3.1520705223083496 seconds --- cached


In [ ]:
# Partition by the "bedrooms" field on the formatted parquet home sales data
output_path = "parquet_csv"
#writing into multiple csv based on the number of bedrooms
df.write.partitionBy("bedrooms").parquet(output_path, mode="overwrite")

In [ ]:
#Read the parquet formatted data.

parquet_df = spark.read.parquet(output_path, header=True)
#print(df.columns)

#show that different numbers of bedrooms exist
print(parquet_df.select("bedrooms").distinct().show())

parquet_df.show()

+--------+
|bedrooms|
+--------+
|     4.0|
|     3.0|
|     5.0|
|    NULL|
|     1.0|
|     2.0|
+--------+

None
+--------------+--------------------+--------------------+--------+----------+--------------+------------------+---------+---------+--------------------+-------------+--------+-------------------+-------+---------+-------+---------+-----------------+-----------+-------------+------+-------------+--------------+------------+-------+----+-----+---------------+---------+-----+-----+--------+
|        region|                city|             quarter|   price|  datetime|parking_spaces|bathrooms_per_room|bathrooms|top_floor|               state|energy_rating|sea_view|centralized_warming|surface|furnished|balcony|TV_system|external_exposure|optic_fiber|electric_gate|cellar|common_garden|private_garden|alarm_system|doorman|pool|villa|entire_property|apartment|attic|lofts|bedrooms|
+--------------+--------------------+--------------------+--------+----------+--------------+-------

In [ ]:
# Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('homesales')

In [ ]:
# Running the query that filters out the location with average price
# Finding the runtime and comparing it to the cached version

start_time = time.time()

query = (
    "SELECT city, ROUND(AVG(price), 0) AS avg_price "
    "FROM homesales "
    "GROUP BY city "
    )


spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+---------+
|                city|avg_price|
+--------------------+---------+
|             Palermo| 180351.0|
|             Codogno|  99500.0|
|       Villa Estense| 103000.0|
|    Quattro Castella| 298333.0|
|   Piano di Sorrento| 440000.0|
|           Spadafora|  94333.0|
|Sant'Agata sul Sa...| 238000.0|
|            Piombino| 160000.0|
|           Viareggio| 479322.0|
|     Forte dei Marmi| 986015.0|
|             Vignate| 318000.0|
|          Roccavione| 135000.0|
|              Spinea| 244815.0|
|         Bentivoglio| 320000.0|
|San Giovanni in P...| 301642.0|
|              Giarre| 103024.0|
|     Castell'Arquato| 390000.0|
|  Riva Presso Chieri| 195000.0|
|      Cassano d'Adda| 181500.0|
|               Carpi| 266629.0|
+--------------------+---------+
only showing top 20 rows

--- 0.9236037731170654 seconds ---


In [ ]:
#  Uncache the home_sales temporary table.

spark.catalog.uncacheTable("homesales")

# Check that the home_sales is no longer cached

spark.catalog.isCached('homesales')

False

In [ ]:
#converting data frame to a pandas df
df_pandas= df.toPandas()
df_pandas

,region,city,quarter,price,datetime,parking_spaces,bathrooms_per_room,bathrooms,bedrooms,top_floor,...,common_garden,private_garden,alarm_system,doorman,pool,villa,entire_property,apartment,attic,lofts
0,Sardegna,Valledoria,Via Alessandro Volta,78000.0,2022-12-17,0,0.3333333333333333,1.0,3.0,0,...,1,0,0,0,0,0,0,1,0,0
1,Sardegna,San Teodoro,None,460000.0,2023-03-31,1,0.6666666666666666,2.0,3.0,1,...,0,0,0,1,1,0,0,1,0,0
2,Sardegna,Cagliari,Fiera - Monte Mixi,387000.0,2023-04-18,0,0.5,2.0,4.0,0,...,0,0,0,0,0,0,0,1,0,0
3,Sardegna,Cagliari,Piazza Giovanni XXIII,257000.0,2023-04-14,0,0.4,2.0,5.0,0,...,0,0,0,0,0,0,0,1,0,0
4,Sardegna,Budoni,Località Baia Sant'Anna,370000.0,2023-03-31,1,0.6666666666666666,2.0,3.0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22322,Toscana,Firenze,Castello,285000.0,2023-06-22,1,0.25,1.0,4.0,0,...,0,0,0,0,0,0,0,1,0,0
22323,Toscana,Pontassieve,None,358000.0,2023-06-28,1,0.6,3.0,5.0,0,...,0,1,0,0,0,1,0,0,0,0
22324,Toscana,Firenze,Piazza della Repubblica,580000.0,2023-08-07,0,0.5,2.0,4.0,1,...,0,0,0,0,0,0,0,1,0,0
22325,Toscana,Marciana,None,240000.0,2023-05-18,1,0.3333333333333333,1.0,3.0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#reset index when neccessary
df_pandas = df_pandas.reset_index(drop=True)


In [ ]:
#reading in csv file because the pandas data frame was not working to par
sales_df = pd.read_csv('sales.csv')
sales_df

<ipython-input-21-959741768bcc>:2: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_df = pd.read_csv('sales.csv')


,region,city,quarter,price,datetime,parking_spaces,bathrooms_per_room,bathrooms,bedrooms,top_floor,...,common_garden,private_garden,alarm_system,doorman,pool,villa,entire_property,apartment,attic,lofts
0,Sardegna,Valledoria,Via Alessandro Volta,78000.0,2022-12-17,0,0.333333,1.0,3.0,0,...,1,0,0,0,0,0,0,1,0,0
1,Sardegna,San Teodoro,NaN,460000.0,2023-03-31,1,0.666667,2.0,3.0,1,...,0,0,0,1,1,0,0,1,0,0
2,Sardegna,Cagliari,Fiera - Monte Mixi,387000.0,2023-04-18,0,0.500000,2.0,4.0,0,...,0,0,0,0,0,0,0,1,0,0
3,Sardegna,Cagliari,Piazza Giovanni XXIII,257000.0,2023-04-14,0,0.400000,2.0,5.0,0,...,0,0,0,0,0,0,0,1,0,0
4,Sardegna,Budoni,Località Baia Sant'Anna,370000.0,2023-03-31,1,0.666667,2.0,3.0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22286,Toscana,Firenze,Castello,285000.0,2023-06-22,1,0.250000,1.0,4.0,0,...,0,0,0,0,0,0,0,1,0,0
22287,Toscana,Pontassieve,NaN,358000.0,2023-06-28,1,0.600000,3.0,5.0,0,...,0,1,0,0,0,1,0,0,0,0
22288,Toscana,Firenze,Piazza della Repubblica,580000.0,2023-08-07,0,0.500000,2.0,4.0,1,...,0,0,0,0,0,0,0,1,0,0
22289,Toscana,Marciana,NaN,240000.0,2023-05-18,1,0.333333,1.0,3.0,0,...,0,0,0,0,0,0,0,1,0,0
